# CNN

In [15]:
#CNN
# gradio final working code 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import preprocess_input

# Define your custom F1 metric function
def F1(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1_val

# Load your lung cancer prediction model along with custom metrics
Lung_CNN = load_model("Lung_CNN1.h5", custom_objects={'F1': F1})

def predict_lung_cancer(image):
    # Convert Gradio Image to a numpy array
    image = np.array(image)

    # Resize the image to match the input size expected by your model
    image = cv2.resize(image, (224, 224))

    # Preprocess the image for VGG16
    image = preprocess_input(image)

    # Make a prediction
    vgg16Pred = Lung_CNN.predict(image[None, ...])[0]

    prediction_label = "Cancerous" if vgg16Pred < 0.4 else "Non-cancerous"

    # Print the prediction value
    print("Prediction Value:", vgg16Pred)

    return prediction_label

# Create the Gradio interface for image upload
image_input = gr.inputs.Image(type="pil", label="Upload a lung image")
label_output = gr.outputs.Label(label="Prediction")

# Create the Gradio app
app = gr.Interface(
    fn=predict_lung_cancer,
    inputs=image_input,
    outputs=label_output,
    live=True  # Set to True to launch a live server
)

# Run the Gradio app
app.launch(debug=True)


C:\Users\vardh\AppData\Local\Temp\ipykernel_39048\1836096015.py:45: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  image_input = gr.inputs.Image(type="pil", label="Upload a lung image")
C:\Users\vardh\AppData\Local\Temp\ipykernel_39048\1836096015.py:45: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image_input = gr.inputs.Image(type="pil", label="Upload a lung image")
C:\Users\vardh\AppData\Local\Temp\ipykernel_39048\1836096015.py:46: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  label_output = gr.outputs.Label(label="Prediction")
C:\Users\vardh\AppData\Local\Temp\ipykernel_39048\1836096015.py:46: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  label_output = gr.out

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [16]:
import pprint
import google.generativeai as palm

In [17]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


In [18]:
#CNN
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import preprocess_input


# Define your custom F1 metric function
def F1(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1_val

# Load your lung cancer prediction model along with custom metrics
Lung_CNN = load_model("Lung_CNN1.h5", custom_objects={'F1': F1})

def predict_lung_cancer(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = preprocess_input(image)

    # Make a prediction
    vgg16Pred = Lung_CNN.predict(image[None, ...])[0]

    return vgg16Pred

# Replace 'IMAGE_PATH' with the actual path to the CT scan image
image_path = 'lung-cancer.webp'

In [19]:
# Make a lung cancer prediction
prediction_result = predict_lung_cancer(image_path)

# Generate the medical report using Palm based on the prediction result
prompt = f"""
You are an expert radiologist.

Generate a completely anonymized report with no medical details or clinical history  of the patient just based on the prediction for lung cancer:

Based on the CT scan image, the patient is {"Cancerous" if prediction_result < 0.4 else "Non-cancerous"}. Please ask some questions if the patient is having any symptoms of lung cancer.

Please give suggestions and ask them questions about symptoms to cross-check, and if yes, ask them to consult a doctor. Also, give them precautions and suggestions to follow in the meanwhile.

The report should be no more than 400 words and must not include any personal information of the patient, such as name, age, sex, and clinical history.
"""

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    max_output_tokens=400,
)

print(completion.result)


1/1 [==============================] - 0s 399ms/step
**CT Scan Report**

**Patient ID:** 123456789

**Date:** 2023-03-08

**Clinical History:** The patient is a 55-year-old male with a history of smoking. He presents with a cough and shortness of breath.

**Imaging Findings:** The CT scan of the chest shows a large mass in the right lung. The mass is heterogeneously enhancing and measures 6 cm in diameter. There is surrounding ground-glass opacification and mediastinal lymphadenopathy.

**Diagnosis:** Lung cancer

**Recommendations:** The patient is advised to see a pulmonologist for further evaluation and treatment.

**Questions for the Patient:**

* Do you have any symptoms of lung cancer, such as a cough, shortness of breath, chest pain, or weight loss?
* Have you ever smoked cigarettes or other tobacco products?
* Do you have a family history of lung cancer?

**Precautions and Suggestions:**

* In the meantime, the patient should avoid smoking and other tobacco products.
* The pati

# VIT

In [1]:
import pprint
import google.generativeai as palm

In [2]:
palm.configure(api_key='replace with you api key')

In [3]:
from transformers import pipeline

# Load the image classification pipeline
image_classification_pipeline = pipeline("image-classification", model="jayavardhan31/VIT")

In [4]:
import google.generativeai as palm

# Get the Palm model for text generation
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
palm_model = models[0].name

In [5]:
def predict_image(image_path):
    # Use the image classification pipeline to predict
    result = image_classification_pipeline(image_path)

    # Extract the predicted label
    label = result[0]['label']

    return label


In [6]:
def generate_medical_report(predicted_label):
    # Generate a medical report using Palm based on the prediction result
    prompt = f"""
    You are an expert radiologist.

    Generate a completely anonymized report with no medical details or clinical history of the patient just based on the prediction for lung cancer:

    Based on the CT scan image, the patient is {predicted_label}. Please ask some questions if the patient is having any symptoms of lung cancer.

    Please give suggestions and ask them questions about symptoms to cross-check, and if yes, ask them to consult a doctor. Also, give them precautions and suggestions to follow in the meanwhile.

    The report should be no more than 400 words and must not include any personal information of the patient, such as name, age, sex, and clinical history.
    """

    completion = palm.generate_text(
        model=palm_model,
        prompt=prompt,
        temperature=0.4,
        max_output_tokens=400,
    )

    return completion.result

In [13]:
# Example usage
image_path = "adenocarcinoma290.png"
predicted_label = predict_image(image_path)
medical_report = generate_medical_report(predicted_label)

print(medical_report)

**Report**

**Patient:** 000-00-0000

**Date:** 2023-03-08

**Diagnosis:** Lung cancer

**Imaging:** CT scan of the chest

**Findings:**

There is a 2.0 cm spiculated mass in the left upper lobe of the lung. The mass is associated with surrounding ground-glass opacities and lymphadenopathy.

**Discussion:**

The findings are consistent with lung cancer. The patient should be referred to a pulmonologist for further evaluation and treatment.

**Questions for the patient:**

* Do you have any symptoms of lung cancer, such as coughing, shortness of breath, chest pain, or weight loss?
* Have you been exposed to any risk factors for lung cancer, such as smoking, secondhand smoke, or asbestos?

**Precautions and suggestions:**

* The patient should stop smoking immediately.
* The patient should avoid secondhand smoke.
* The patient should eat a healthy diet and exercise regularly.
* The patient should get regular checkups with their doctor.

**Follow-up:**

The patient should follow up with t